In [1]:
import csv
import ipyparams
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import random
import re
import shutil
import string  
import sys
import tensorflow as tf
import tensorflow_hub as hub

from collections import Counter
# from google_trans_new import google_translator
from joblib import Parallel, delayed
from nltk.corpus import stopwords
from scipy import stats
from sklearn.feature_extraction import text
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, models, layers, regularizers, preprocessing, datasets, metrics, losses, optimizers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorboard.plugins.hparams import api as hp 

base_path = os.path.abspath('/tf/eipi10/jian-xu3/snps-classification')
sys.path.append(base_path)

import snps 

from snps.core import text_predictor 
from snps.core import util 
from snps.utils import data
from snps.core.text_classification import lr_schedule
from snps.core.text_classification import plot_distribution, plot_length_distribution, plot_frequency_distribution
from snps.core.text_classification import Params, TextClassificationHelper
from snps.core.text_classification import SimpleTextDatasets, SequenceTextDatasets, RawTextDatasets,TransferTextDatasets

<IPython.core.display.Javascript object>

2021-07-05 08:45:26,327: INFO: deployment = dev
2021-07-05 08:45:26,329: INFO: dataset_name = snps_18510_c8
2021-07-05 08:45:26,330: INFO: working_path = /tf/eipi10/jian-xu3/snps-classification


In [2]:
#!/usr/bin/env python3

import json
import os
import os.path
import posixpath
import subprocess
import pandas as pd
import psutil
import requests
from IPython.display import display


def show_notebooks_table(host, port):
    """Show table with info about running jupyter notebooks.

    Args:
        host: host of the jupyter server.
        port: port of the jupyter server.

    Returns:
        DataFrame with rows corresponding to running notebooks and following columns:
            * index: notebook kernel id.
            * path: path to notebook file.
            * pid: pid of the notebook process.
            * memory: notebook memory consumption in percentage.
    """
    pd.set_option('display.max_colwidth', 300)
    
    notebooks = get_running_notebooks(host, port)
    prefix = long_substr([notebook['path'] for notebook in notebooks])
    df = pd.DataFrame(notebooks)
    df = df.set_index('kernel_id')
    df.index.name = prefix
    df.path = df.path.apply(lambda x: x[len(prefix):])
    df['pid'] = df.apply(lambda row: get_process_id(row.name), axis=1)
    # same notebook can be run in multiple processes
    df = expand_column(df, 'pid')
    df['memory'] = df.pid.apply(memory_usage_psutil)
    df['cpu'] = df.pid.apply(cpu_usage_psutil)
    df['num_threads'] = df.pid.apply(num_threads_psutil)    
    df = df.sort_values('memory', ascending=False)
    df.index = range(1, len(df)+1)
    return df


def get_running_notebooks(host, port, password='xxw'):
    """
    Get kernel ids and paths of the running notebooks.

    Args:
        host: host at which the notebook server is listening. E.g. 'localhost'.
        port: port at which the notebook server is listening. E.g. 8888.
            
    Returns:
        list of dicts {kernel_id: notebook kernel id, path: path to notebook file}.
    """
        
    BASE_URL = 'http://{0}:{1}/'.format(host, port)
        
    # Get the cookie data
    s = requests.Session()
    url = BASE_URL + 'login?next=%2F'
    resp = s.get(url)
    xsrf_cookie = resp.cookies['_xsrf']

    # Login with the password
    params = {'_xsrf': xsrf_cookie, 'password': password}
    res = s.post(url, data=params)

    # Find which kernel corresponds to which notebook
    # by querying the notebook server api for sessions
    url = posixpath.join(BASE_URL, 'api', 'sessions')
    ret = s.get(url)
    #print('Status code:', ret.status_code)

    # Get the notebook list
    res = json.loads(ret.text)
    notebooks = [{'kernel_id': notebook['kernel']['id'],
                  'path': notebook['notebook']['path']} for notebook in res]
    return notebooks


def get_process_id(name):
    """Return process ids found by (partial) name or regex.

    Source: https://stackoverflow.com/a/44712205/304209.
    >>> get_process_id('kthreadd')
    [2]
    >>> get_process_id('watchdog')
    [10, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61]  # ymmv
    >>> get_process_id('non-existent process')
    []
    """
    child = subprocess.Popen(['pgrep', '-f', name], stdout=subprocess.PIPE, shell=False)
    response = child.communicate()[0]
    return [int(pid) for pid in response.split()]


def memory_usage_psutil(pid=None):
    """Get memory usage percentage by current process or by process specified by id, like in top.

    Source: https://stackoverflow.com/a/30014612/304209.

    Args:
        pid: pid of the process to analyze. If None, analyze the current process.

    Returns:
        memory usage of the process, in percentage like in top, values in [0, 100].
    """
    if pid is None:
        pid = os.getpid()
    process = psutil.Process(pid)
    return round(process.memory_percent(),3)

def cpu_usage_psutil(pid=None):
    """Get cpu usage percentage by current process or by process specified by id, like in top.

    Source: https://stackoverflow.com/a/30014612/304209.

    Args:
        pid: pid of the process to analyze. If None, analyze the current process.

    Returns:
        cpu usage of the process, in percentage like in top, values in [0, 100].
    """
    if pid is None:
        pid = os.getpid()
    process = psutil.Process(pid)
    return round(process.cpu_percent(),3)


def num_threads_psutil(pid=None):
    """Get num_threads by current process or by process specified by id, like in top.

    Source: https://stackoverflow.com/a/30014612/304209.

    Args:
        pid: pid of the process to analyze. If None, analyze the current process.

    Returns:
        num_threads of the process
    """
    if pid is None:
        pid = os.getpid()
    process = psutil.Process(pid)
    return process.num_threads()


def long_substr(strings):
    """Find longest common substring in a list of strings.

    Source: https://stackoverflow.com/a/2894073/304209.

    Args:
        strings: list of strings.

    Returns:
        longest substring which is found in all of the strings.
    """
    substr = ''
    if len(strings) > 1 and len(strings[0]) > 0:
        for i in range(len(strings[0])):
            for j in range(len(strings[0])-i+1):
                if j > len(substr) and all(strings[0][i:i+j] in x for x in strings):
                    substr = strings[0][i:i+j]
    return substr


def expand_column(dataframe, column):
    """Transform iterable column values into multiple rows.

    Source: https://stackoverflow.com/a/27266225/304209.

    Args:
        dataframe: DataFrame to process.
        column: name of the column to expand.

    Returns:
        copy of the DataFrame with the following updates:
            * for rows where column contains only 1 value, keep them as is.
            * for rows where column contains a list of values, transform them
                into multiple rows, each of which contains one value from the list in column.
    """
    tmp_df = dataframe.apply(
        lambda row: pd.Series(row[column]), axis=1).stack().reset_index(level=1, drop=True)
    tmp_df.name = column
    return dataframe.drop(column, axis=1).join(tmp_df)

df = show_notebooks_table('15.15.166.35', 18888)
display(df)

,path,pid,memory,cpu,num_threads
1,eipi10/xuxiangwen.github.io/_notes/00.ipynb,47,0.377,0.0,8


In [3]:
import ast
import types
import astunparse

func_def = \
"""
def add(x, y):
    return x + y
print(add(3, 5))
"""

cm = compile(func_def, '<string>', 'exec')
assert(isinstance(cm, types.CodeType))

exec(cm)

8


In [4]:
# 生成AST
r_node = ast.parse(func_def)
# 也可调用ast.dump(r_node)，但它不能进行缩进输出
ast_output = astunparse.dump(r_node)  
print(ast_output)

# 从AST得到源代码
print('-'*50)
print(astunparse.unparse(r_node))

Module(body=[
  FunctionDef(
    name='add',
    args=arguments(
      args=[
        arg(
          arg='x',
          annotation=None),
        arg(
          arg='y',
          annotation=None)],
      vararg=None,
      kwonlyargs=[],
      kw_defaults=[],
      kwarg=None,
      defaults=[]),
    body=[Return(value=BinOp(
      left=Name(
        id='x',
        ctx=Load()),
      op=Add(),
      right=Name(
        id='y',
        ctx=Load())))],
    decorator_list=[],
    returns=None),
  Expr(value=Call(
    func=Name(
      id='print',
      ctx=Load()),
    args=[Call(
      func=Name(
        id='add',
        ctx=Load()),
      args=[
        Num(n=3),
        Num(n=5)],
      keywords=[])],
    keywords=[]))])
--------------------------------------------------


def add(x, y):
    return (x + y)
print(add(3, 5))



In [5]:
class CodeVisitor(ast.NodeVisitor):
    def visit_BinOp(self, node):
        if isinstance(node.op, ast.Add):
            node.op = ast.Sub()
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        print('Function Name:%s'% node.name)
        self.generic_visit(node)
        func_log_stmt = ast.print(
            dest = None,
            values = [ast.Str(s = 'calling func: %s' % node.name, lineno = 0, col_offset = 0)],
            nl = True,
            lineno = 0,
            col_offset = 0,
        )
        node.body.insert(0, func_log_stmt)

r_node = ast.parse(func_def)
visitor = CodeVisitor()
visitor.visit(r_node)

print(astunparse.unparse(r_node))
exec(compile(r_node, '<string>', 'exec'))

Function Name:add


AttributeError: module 'ast' has no attribute 'print'

In [ ]:
!pip install -U ast


In [ ]:
dir(ast)

In [ ]:
import collections, re
pairs = collections.defaultdict(int)
pairs['\u2581 3', '4']=5
pairs['4', '5']=3
pairs

In [ ]:
def merge_vocab(pair):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')

    w_out = p.sub(''.join(pair), word)
    return w_out

bigrams = [re.escape(' '.join(pair)) for pair in pairs]
print(f'bigrams={bigrams}')
ps = [re.compile(r'(?<!\S)' + bigram + r'(?!\S)') for bigram in bigrams] 
print(f'ps={ps}')

In [ ]:
pair = list(pairs.keys())[0]
bigram = re.escape(' '.join(pair))
p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
print(p)
word=r'3\\ 4'
p.sub(''.join(pair), word)

In [ ]:
''.join(pair)

In [ ]:
word='3\\ 4'
word 

In [ ]:
word='3\\ 4'
word

In [ ]:
def sub(pair, word):
    '''在word中替换pair为pair去空格'''
    print(f'word = {word}')
    bigram = re.escape(' '.join(pair))

    # 匹配，前后都为空的pair
    # (?<!\S)	匹配前面不是非空的位置，即匹配前面是空
    # (?!\S)	匹配后面跟的不是非空的位置，即匹配后面是空      
    pattern = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    print(f'pattern = {pattern}')
    new_word = pattern.sub(''.join(pair), word)
    print(f'new_word = {new_word}') 

word = 'b c ab c b c c bd b c'

pair = ('b', 'c')
# 从结果可以看到，第一个,第三个，第五个b c被替换成了bc
sub(pair, word)

In [ ]:

# word = re.escape('\u2581abcd') 
# print(word)


In [ ]:
import re

pattern = '\S'
string = ' 344-Hello--World!'
result = re.search(pattern, string)
print(result)
print(result.group())

In [ ]:
pattern = '[a-z]+'
string = '-----2344-HELLO--WORLD!'
result = re.search(pattern, string)
print(result)

In [ ]:
pair = ('4', '5')
bigram = re.escape(' '.join(pair))
p = re.compile(bigram)
print(p)
word = re.escape('1234 567') 
print(word)
print(''.join(pair))
print(p.sub(''.join(pair), word))

In [ ]:
import numpy as np

x1 = np.array([0.2, 0.3, 0.1, 0.4])
scores = np.array([2, 3, 4, -1])
order = scores.argsort()[::-1] 
print(order)
i = order[0]
np.maximum(x1[i], x1[order[1:]])

In [ ]:
def py_cpu_nms(dets, thresh):
    """
    dets是shape为 Nx5 的ndarray,如:
    dets = np.array([[5,5,20,20,0.7],
                     [0,0,10,10,0.3]])
    
    """
    x1 = dets[:, 0] #所有的x1
    y1 = dets[:, 1] #所有的y1
    x2 = dets[:, 2] #所有的x2
    y2 = dets[:, 3] #所有的y2
    scores = dets[:, 4] #所有的得分

    areas = (x2 - x1 + 1) * (y2 - y1 + 1) #所有框的面积
    order = scores.argsort()[::-1] #面积排序

    keep = [] #
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        
        print(x1[order[1:]])
        print(xx1,yy1,xx2,yy2)

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h

        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

In [ ]:
import numpy as np


In [ ]:
a =1


In [ ]:
!pip install  jupyter_contrib_nbextensions
!pip install jupyter_nbextensions_configurator

In [ ]:
!jupyter contrib nbextension install --user 
!jupyter nbextensions_configurator enable --user

In [ ]:
1

In [ ]:
a=1

In [ ]:
import numpy as np

In [ ]:
np.testing.

In [6]:
561*728*5

2042040

In [7]:

class Dog:
 
    def bark(self):
        print("汪汪叫")
 
# 子类  继承
class XiaoTianQuan(Dog):
 
    def fly(self):
        print("我会飞")
 
    # 可以重写父类中的同名方法
    def bark(self):
 
        # super().父类方法名 调用父类中的方法 (第一种方式)(推荐)
        super().bark()
 
        # 父类名.方法(self) 调用父类中的方法 (第二种方式,python2.x)(不推荐,父类名修改后,此处也得改)
        Dog.bark(self)
 
        # 注意：如果使用子类名调用方法，可能会出现递归调用 -- 死循环！
        # XiaoTianQuan.bark(self)  # 会产生死循环
 
        # 针对子类特有的需求，进行扩展
        print("神一样的叫唤...")
 
 
xtq = XiaoTianQuan()
xtq.bark()


汪汪叫
汪汪叫
神一样的叫唤...


In [14]:
a = lambda x, y: x+y, x-y

NameError: name 'x' is not defined

In [13]:
a(2, 3)

(5, -1)

In [16]:
r= np.array([[1, 2, 3], [3, 4, 2], [0, 0, 0]])

In [18]:
r[np.all(r == 0, axis=1)]

array([[0, 0, 0]])

In [19]:
r[~np.all(r == 0, axis=1)]

array([[1, 2, 3],
       [3, 4, 2]])

In [2]:
!pwd

/tf/eipi10/xuxiangwen.github.io/_notes


In [9]:
import os
dataset_path='/tf/eipi10/xuxiangwen.github.io/_notes/'


'_notes'

In [17]:
def f2(a, b=2, c=3):
    return a + b + c

In [29]:
a = ['1', '2', '3']
a[1:2]

['2']

In [ ]:
class TextReader:
    """Print and number lines in a text file."""

    def __init__(self, filename):
        self.filename = filename
        self.file = open(filename)
        self.lineno = 0

    def readline(self):
        self.lineno += 1
        line = self.file.readline()
        if not line:
            return None
        if line.endswith('\n'):
            line = line[:-1]
        return "%i: %s" % (self.lineno, line)

    def __getstate__(self):
        # Copy the object's state from self.__dict__ which contains
        # all our instance attributes. Always use the dict.copy()
        # method to avoid modifying the original state.
        state = self.__dict__.copy()
        # Remove the unpicklable entries.
        del state['file']
        return state

    def __setstate__(self, state):
        # Restore instance attributes (i.e., filename and lineno).
        self.__dict__.update(state)
        # Restore the previously opened file's state. To do so, we need to
        # reopen it and read from it until the line count is restored.
        file = open(self.filename)
        for _ in range(self.lineno):
            file.readline()
        # Finally, save the file.
        self.file = file
        
reader = TextReader("hello.txt")
print(reader.readline())
print(reader.readline())
s = pickle.dumps(reader)
#print(s)
new_reader = pickle.loads(s)
print(new_reader.readline())

In [30]:
!pwd

/tf/eipi10/xuxiangwen.github.io/_notes


In [31]:
import os
os.path.dirname('/tf/eipi10/xuxiangwen.github.io/_notes')

'/tf/eipi10/xuxiangwen.github.io'

In [43]:
a = {'a':1, 'b':2}
class B:
    def __init__(self):
        self.c = {'e':2, 'a':3}

dict(a, **B().c)

{'a': 3, 'b': 2, 'e': 2}

In [42]:
b.c

AttributeError: 'dict' object has no attribute 'c'

In [2]:
60000/32

1875.0